In [ ]:
%reset -fs
import pandas as pd
import  numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from dask import dataframe as dd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.options.mode.use_inf_as_na = True

In [ ]:
#Dask !!!
ddf = dd.read_csv(r"C:\Users\nated\Downloads\CSV-01-12\01-12\DrDoS_LDAP.csv", dtype={'SimillarHTTP': 'object'},blocksize='64MB')
df = ddf.compute()

In [ ]:
df.columns = df.columns.str.replace(' ', '')
df.columns

In [ ]:
(df['Label'].value_counts()) / len(df) * 100

In [ ]:
df.drop(columns=['FlowID', 'SourceIP', 'DestinationIP', 'Timestamp', 'SimillarHTTP', 'SourcePort', 'DestinationPort'], inplace=True)
df

In [ ]:
df.dropna(inplace=True)

In [ ]:
cols = df.drop(columns=['Label']).columns.tolist()
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

In [ ]:
X = df.drop(columns=["Label"])
y = df.Label

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', max_iter=1000000, n_jobs=-1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = .2, stratify=y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25 stratify=y_train_val)

In [ ]:
(y_train.value_counts()) / len(y_train) * 100

In [ ]:
X_train_scaled = pd.DataFrame(
    std.fit_transform(X_train),
    columns = X_train.columns)

In [ ]:
import joblib
from dask.distributed import Client

client = Client(processes=False)

In [ ]:
with joblib.parallel_backend("dask"):
    lr.fit(X_train_scaled, y_train)
    

In [ ]:
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, fbeta_score, confusion_matrix

In [ ]:
X_val_scaled = pd.DataFrame(std.transform(X_val), columns = X_val.colunmns)

In [ ]:
y_val_preds = lr.predict(X_val_scaled)
print("With default decision threshold:")
print("Precision: {}, Recall: {}".format(precision_score(y_val, y_val_preds, pos_label='BENIGN'), recall_score(y_val, y_val_preds, pos_label='BENIGN')))

In [ ]:
import seaborn as sns

In [ ]:
lr_confusion = confusion_matrix(y_val, y_val_preds)
sns.heatmap(lr_confusion, cmap=plt.cm.Blues, annot=True, square=True)
plt.xlabel('Predicted Traffic Type')
plt.ylabel('Actual Traffic Type')
plt.title('Logistic Regression Confusion Matrix')